<a href="https://colab.research.google.com/github/BertrandKafando/MachineLearning/blob/master/A_CNN_for_facial_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present
TrainingImagePath='/content/drive/MyDrive/Face Images/Final Training Images'

from keras.preprocessing.image import ImageDataGenerator
# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')


# Generating the Testing Data
validation_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
validation_set.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [ ]:
training_set.image_shape

(64, 64, 3)

In [ ]:
training_set.

Creating a mapping for index and face names

In [ ]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


Creating the CNN face recognition model
In the below code snippet, I have created a CNN model with

2 hidden layers of convolution
2 hidden layers of max pooling
1 layer of flattening
1 Hidden ANN layer
1 output layer with 16-neurons (one for each face)
You can increase or decrease the convolution, max pooling, and hidden ANN layers and the number of neurons in it.

Just keep in mind, the more layers/neurons you add, the slower the model becomes.

In [ ]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=len(training_set),
                    epochs=10,
                    validation_data=validation_set,
                    validation_steps=10)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

<ipython-input-42-21f59c7b6904>:44: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 77.2418 - accuracy: 0.0820

8/8 [==============================] - 6s 671ms/step - loss: 77.2418 - accuracy: 0.0820 - val_loss: 3.0094 - val_accuracy: 0.0697
Epoch 2/10
8/8 [==============================] - 3s 406ms/step - loss: 2.6705 - accuracy: 0.1803
Epoch 3/10
8/8 [==============================] - 3s 391ms/step - loss: 1.9684 - accuracy: 0.4057
Epoch 4/10
8/8 [==============================] - 3s 395ms/step - loss: 1.2930 - accuracy: 0.6230
Epoch 5/10
8/8 [==============================] - 3s 406ms/step - loss: 0.5583 - accuracy: 0.8607
Epoch 6/10
8/8 [==============================] - 3s 406ms/step - loss: 0.3125 - accuracy: 0.8770
Epoch 7/10
8/8 [==============================] - 3s 401ms/step - loss: 0.2354 - accuracy: 0.9385
Epoch 8/10
8/8 [==============================] - 3s 400ms/step - loss: 0.2067 - accuracy: 0.9344
Epoch 9/10
8/8 [==============================] - 3s 395ms/step - loss: 0.2065 - accuracy: 0.9590
Epoch 10/10
8/8 [==============================] - 3s 395ms/step - loss: 0.1518 - accu

In [ ]:
'''########### Making single predictions ###########'''
import numpy as np
from tensorflow.keras.preprocessing import image

ImagePath='/content/drive/MyDrive/Face Images/Final Testing Images/face16/3face16.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)], "   |   Expected/reality : ", " face16")

########################################
Prediction is:  face16    |   Expected/reality :   face16


In [ ]:
'''########### Making single predictions ###########'''

import numpy as np
import os
from tensorflow.keras.preprocessing import image

s = 0
v = 0
path='/content/drive/MyDrive/Face Images/Final Testing Images'
for file in os.listdir(path):
  # get image path
  if file!=".DS_Store":
    filePath=os.path.join(path,file)
    for im in os.listdir(filePath): 
      ImagePath=os.path.join(filePath,im)
      test_image=image.load_img(ImagePath,target_size=(64, 64))
      test_image=image.img_to_array(test_image)
      test_image=np.expand_dims(test_image,axis=0)
      result=classifier.predict(test_image,verbose=0) 
      print('####'*10)
      print(f'Real:{im[1:]} Prediction: {ResultMap[np.argmax(result)]}')
      s+=1
      if(im[1:].split(".")[0] == ResultMap[np.argmax(result)]):
        v+=1

print(v)
print(s)
print(f'Accuracy: {v/s}')

########################################
Real:face1.jpg Prediction: face14
########################################
Real:face1.jpg Prediction: face1
########################################
Real:face1.jpg Prediction: face1
########################################
Real:face1.jpg Prediction: face14
########################################
Real:face14.jpg Prediction: face14
########################################
Real:face14.jpg Prediction: face14
########################################
Real:face14.jpg Prediction: face14
########################################
Real:face14.jpg Prediction: face14
########################################
Real:face15.jpg Prediction: face15
########################################
Real:face15.jpg Prediction: face15
########################################
Real:face15.jpg Prediction: face15
########################################
Real:face15.jpg Prediction: face15
########################################
Real:face16.jpg Prediction: face16
##################